In [1]:
!pip3 install --upgrade --user google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 15.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [2]:
PROJECT_ID = "vertext-ai-dar"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [36]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

In [110]:
def multiply(a:float, b:float):
    """returns a * b."""
    print("Calling Multiply function")
    return a * b

def add(a:float, b:float):
    """returns a + b."""
    print("Calling Add function")
    return a + b


multiply_info = FunctionDeclaration(
    name="multiply",
    description="Multiplies two numbers and returns the result",
    parameters={
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"}
        },
    },
)

add_info = FunctionDeclaration(
    name="add",
    description="Adds two numbers and returns the result",
    parameters={
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"}
        },
    },
)


math_tool = Tool(
    function_declarations=[
        multiply_info,
        add_info
    ],
)

model = GenerativeModel(
    "gemini-1.5-pro-001",
    system_instruction=["""Answer the user's question,
    but do not do any math yourself."""],
    tools=[math_tool]
  )

In [111]:
def handle_response(response):

  # If there is a function call then invoke it
  # Otherwise print the response.
  if response.candidates[0].function_calls:
    function_call = response.candidates[0].function_calls[0]
  else:
    print(response.text)
    return


  if function_call.name == "multiply":
      # Extract the arguments to use in your function
      a = function_call.args["a"]
      b = function_call.args["b"]

      # Call your function
      result = multiply(a, b)

      # Send the result back to the model
      response = chat.send_message("{0}".format(result))

      # Recursive call
      handle_response(response)

  elif function_call.name == "add":
      # Extract the arguments to use in your function
      a = function_call.args["a"]
      b = function_call.args["b"]

      # Call your function
      result = add(a, b)

      # Send the result back to the model
      response = chat.send_message("{0}".format(result))
      # Recursive call
      handle_response(response)
  else:
      # Shouldn't get here
      print(function_call)


In [126]:
chat = model.start_chat()

In [127]:
response = chat.send_message("Tell me a joke?")
handle_response(response)

Why did the four drop out of school? 

Because it was too tired of being divided by two! 



In [128]:
response = chat.send_message("I have 7 pizzas each with 16 slides. How many slices do I have?")
handle_response(response)

Calling Multiply function
You have 112 slices of pizza. 



In [129]:
response = chat.send_message("""
Doug brought 3 pizzas.
Andrew brought 4 pizzas.
How many pizzas did they bring together?
""")

handle_response(response)

Calling Add function
Together they brought 7 pizzas. 



In [130]:
response = chat.send_message("""
Doug brought 3 pizzas.
Andrew brought 4 pizzas.
There are 16 slices per pizza.
How many slices are there?
""")

handle_response(response)

Calling Add function
Calling Multiply function
There are 112 slices in total. 

